In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv("azure.env")

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
 api_version=os.getenv("AZURE_OPENAI_VERSION")
)

CHAT_COMPLETIONS_MODEL = os.getenv('CHAT_COMPLETION_MODEL')

prompt = "### Postgres SQL tables, with their properties:\n#\n# Employee(id, name, department_id)\n# Department(id, name, address)\n# Salary_Payments(id, employee_id, amount, date)\n#\n### A query to list the names of the departments which employed more than 10 employees in the last 3 months\n\n query: "
print(prompt)
response = client.chat.completions.create(
  model=CHAT_COMPLETIONS_MODEL,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content": prompt}],
  temperature=0,
  max_tokens=150,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])
print(response.choices[0].message.content)


### Postgres SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### A query to list the names of the departments which employed more than 10 employees in the last 3 months

 query: 
To list the names of the departments which employed more than 10 employees in the last 3 months, you can use the following SQL query:

```sql
WITH RecentEmployees AS (
    SELECT e.department_id
    FROM Employee e
    JOIN Salary_Payments sp ON e.id = sp.employee_id
    WHERE sp.date >= CURRENT_DATE - INTERVAL '3 months'
    GROUP BY e.id, e.department_id
)
SELECT d.name
FROM Department d
JOIN (
    SELECT department_id, COUNT(DISTINCT employee_id) AS employee_count
    FROM RecentEmployees
    GROUP BY department_id
    HAVING COUNT(DISTINCT employee_id) > 10
) AS dept_counts ON d.id = dept_counts.department


In [2]:
code = "Can you explain what does this code do?\n#\n# ###\n\
   Code:\n\
   SELECT d.name FROM Department d JOIN Employee e ON d.id = e.department_id WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months') GROUP BY d.name HAVING COUNT(*) > 10\n#\n#\
   Answer:\n# "

response = client.chat.completions.create(
  model=CHAT_COMPLETIONS_MODEL,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content": code}],
  temperature=0,
  max_tokens=500,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])
print(response.choices[0].message.content)


Certainly! Let's break down the SQL query step by step to understand what it does:

1. **FROM Department d JOIN Employee e ON d.id = e.department_id**:
   - This part of the query joins the `Department` table (aliased as `d`) with the `Employee` table (aliased as `e`) based on the condition that the `id` field in the `Department` table matches the `department_id` field in the `Employee` table. This effectively links each employee to their respective department.

2. **WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months')**:
   - This subquery selects `employee_id` from the `Salary_Payments` table where the `date` of the payment is within the last 3 months (i.e., greater than the current date minus 3 months).
   - The main query then filters the joined `Department` and `Employee` records to include only those employees whose `id` is in the list of `employee_id` values returned by the subquery. This means we are only considering employees who hav